In [1]:
include("../../src/GaPSE.jl");

In [2]:
using DelimitedFiles, Plots, Dierckx, SpecialFunctions, Printf, LaTeXStrings

In [ ]:
FILE_F_MAP = "../../test/datatest/F_REFERENCE.txt"
FILE_PS = "../../test/datatest/file_pk.txt"
FILE_BACKGROUND = "../../test/datatest/WideA_ZA_background.dat"

z_min = 0.05
z_max = 0.20
θ_max = π / 2.0
params = GaPSE.CosmoParams(z_min, z_max, θ_max;
  k_min = 1e-8, k_max = 10.0,
  Ω_b = 0.0489, Ω_cdm = 0.251020, h_0 = 0.70,
    N = 1024, fit_min = 0.05, fit_max= 0.5, con=true)

cosmo = GaPSE.Cosmology(params, FILE_BACKGROUND, FILE_PS, FILE_F_MAP; expand=false)

GaPSE.parameters_used(stdout, cosmo)

In [ ]:
mattia_table = readdlm("/Users/matteofoglieni/Dropbox/GR_effects/Mattia_Output/Monopoles_without_window/xi_lensing_doppler_L0.txt")
mattia_ss = convert(Vector{Float64}, mattia_table[2:end, 1]);
mattia_lensingdopplers = convert(Vector{Float64}, mattia_table[2:end, 2]);

In [ ]:
GaPSE.print_map_ξ_multipole(cosmo, "xi_lensingdoppler_L0.txt", "lensing_doppler"; 
    use_windows = false, N_χs = 150, N_μs = 30)

In [ ]:
my_table = readdlm("xi_lensingdoppler_L0.txt", comments=true);
my_ss = convert(Vector{Float64}, my_table[:,1]);
my_lensingdopplers = convert(Vector{Float64}, my_table[:,2]);
#casto_table = readdlm("tab_kappa_terms.dat")
#casto_ss = convert(Vector{Float64}, casto_table[3:end, 1]);
#casto_lensings = convert(Vector{Float64}, casto_table[3:end, 2]);

In [ ]:
my_xs = my_ss
my_ys = my_lensingdopplers
xs = mattia_ss
ys = mattia_lensingdopplers
spline = Spline1D(xs, ys)
K_VLINE = 150

p1 = plot([my_xs, xs], [my_ys,  ys], #xaxis=:log, #yaxis=:log, 
    lc=[:red :blue], label=["My LensingDoppler" "Mattia LensingDoppler"], 
    ls=[:dash :solid], legend=:bottomleft)
vline!(p1,[K_VLINE], lc=:black, ls=:dash)


p2 = plot(my_xs, spline.(my_xs) .- my_ys, 
    xaxis=:log,label="Difference", lc=:green, legend=:bottomright)
vline!(p2,[K_VLINE], lc=:black, ls=:dash)

p3 = plot(my_xs, spline.(my_xs) ./ my_ys, label="Ratio", lc=:green, xaxis=:log,
    #ylim=(0.99,1.05)
)
vline!(p3,[K_VLINE], lc=:black, ls=:dash)

plot(p1, p2, p3, size=(650, 600), layout= @layout [ x ; y z ] )

In [ ]:
my_xs = my_ss
my_ys = my_lensingdopplers
xs = mattia_ss
ys = mattia_lensingdopplers
spline = Spline1D(xs, ys)

S_VLIM = cosmo.s_eff
S_VLIM_F = @sprintf("%.2f", S_VLIM)

p1 = plot([my_xs, xs], [my_ys,  ys], xaxis=:log, #yaxis=:log, 
    lc=[:red :blue], 
    label=["My LensingDoppler" "Mattia LensingDoppler"], ls=[:dash :solid], 
    legend=:topleft)
vline!(p1, [S_VLIM], label="$S_VLIM_F Mpc/h_0", lc=:black, lw=0.5, ls=:dash)

p2 = plot(my_xs, spline.(my_xs) ./ my_ys, label="Ratio", lc=:green, xaxis=:log,
    ylim=(0.99,1.01), legend=:topleft,
    xlabel=L"$s \quad [h_0^{-1}\,\mathrm{Mpc}]$")
vline!(p2, [S_VLIM], label="$S_VLIM_F Mpc/h_0", lc=:black, lw=0.5, ls=:dash)

P = plot(p1, p2, size=(550, 500), layout= @layout [ x ; z ] )
savefig(P, "xi_LensingDoppler_L0.pdf")
display(P)

In [ ]:
mattia_table_2 = readdlm("/Users/matteofoglieni/Dropbox/GR_effects/Mattia_Output/Monopoles_without_window/xi_doppler_lensing_L0.txt")
mattia_2_ss = convert(Vector{Float64}, mattia_table_2[2:end, 1]);
mattia_2_localgpintegrateds = convert(Vector{Float64}, mattia_table_2[2:end, 2]);

In [ ]:
GaPSE.print_map_ξ_multipole(cosmo,"xi_doppler_lensing_L0.txt",
    "doppler_lensing"; use_windows = false, N_μs = 50)

In [ ]:
my_table_2 = readdlm("xi_doppler_lensing_L0.txt", comments=true);
my_2_ss = convert(Vector{Float64}, my_table_2[:,1]);
my_2_localgpintegrateds = convert(Vector{Float64}, my_table_2[:,2]);

In [ ]:
my_xs = my_2_ss
my_ys = my_2_localgpintegrateds
xs = mattia_2_ss
ys = mattia_2_localgpintegrateds
spline = Spline1D(xs, ys)
K_VLINE = 150

p1 = plot([my_xs, xs], [my_ys,  ys], xaxis=:log, #yaxis=:log, 
    lc=[:red :blue], label=["My DopplerLensing" "Mattia DopplerLensing"], 
    ls=[:dash :solid], legend=:topleft)
vline!(p1,[K_VLINE], lc=:black, ls=:dash)


p2 = plot(my_xs, spline.(my_xs) .- my_ys, 
    xaxis=:log,label="Difference", lc=:green, legend=:bottomright)
vline!(p2,[K_VLINE], lc=:black, ls=:dash)

p3 = plot(my_xs, spline.(my_xs) ./ my_ys, label="Ratio", lc=:green, xaxis=:log,
    #ylim=(0.99,1.05)
)
vline!(p3,[K_VLINE], lc=:black, ls=:dash)

plot(p1, p2, p3, size=(650, 600), layout= @layout [ x ; y z ] )

In [ ]:
my_xs = my_2_ss
my_ys = my_2_localgpintegrateds
xs = mattia_2_ss
ys = mattia_2_localgpintegrateds
spline = Spline1D(xs, ys)

S_VLIM = cosmo.s_eff
S_VLIM_F = @sprintf("%.2f", S_VLIM)

p1 = plot([my_xs, xs], [my_ys,  ys], xaxis=:log, #yaxis=:log, 
    lc=[:red :blue], 
    label=["My DopplerLensing" "Mattia DopplerLensing"], 
    ls=[:dash :solid], legend=:bottomleft)
vline!(p1, [S_VLIM], label="$S_VLIM_F Mpc/h_0", lc=:black, lw=0.5, ls=:dash)

p2 = plot(my_xs, spline.(my_xs) ./ my_ys, label="Ratio", lc=:green, xaxis=:log,
    ylim=(0.99,1.01), legend=:topleft,
    xlabel=L"$s \quad [h_0^{-1}\,\mathrm{Mpc}]$")
vline!(p2, [S_VLIM], label="$S_VLIM_F Mpc/h_0", lc=:black, lw=0.5, ls=:dash)

P = plot(p1, p2, size=(550, 500), layout= @layout [ x ; z ] )
savefig(P, "xi_doppler_lensing_L0.pdf")
display(P)